In [1]:
device = "cuda"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
zip_path= '/content/drive/My Drive/ISPP_Folder/CIFAKE_Shadow_Datasets.zip'

In [5]:
import zipfile
import os

# Define the destination directory for unzipped files
extract_to = '/content/unzipped_folder/'

# Ensure the destination directory exists
os.makedirs(extract_to, exist_ok=True)

# Open and extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files unzipped successfully.")

Files unzipped successfully.


In [6]:
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import datasets

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
])


In [8]:
train_dataset = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Train_Shadow/Dataset5", transform=transform)

In [9]:
assert len(train_dataset) == 10000

In [10]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [11]:
test_dataset = datasets.ImageFolder(root="/content/unzipped_folder/CIFAKE_Shadow_Datasets/New_Test_Shadow/Dataset5",transform=transform)

In [12]:
assert len(test_dataset) == 2000

In [13]:
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.tanh(self.conv1(x)))
        x = self.pool(self.tanh(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor for the fully connected layer
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
shadow_model = CNN()
criterion = nn.CrossEntropyLoss()
shadow_optimizer = optim.Adam(shadow_model.parameters(), lr=0.001)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

shadow_model = shadow_model.to(device)

cuda


In [17]:
import wandb

# Initialize wandb
wandb.init(
    project="Shadow_Model_Training",  # Replace with your project name
    config={
        "epochs": 20,
        "batch_size": train_loader.batch_size,
        "learning_rate": shadow_optimizer.param_groups[0]['lr'],
        "target_epsilon": 0,
        "Run": 2 # Assuming all param groups have the same LR
    }
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: prahalac (prahalac-carnegie-mellon-university). Use `wandb login --relogin` to force relogin


In [18]:
def train(model,optimizer,loader,epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    for i, (images, labels) in enumerate(loader):

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

            # Append predictions and labels for metric calculation
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    # Print the loss after each epoch
    accuracy = 100 * correct / total
    wandb.log({"train_loss": running_loss / len(loader), "train_accuracy": accuracy, "Epoch" : epoch+1})
    print(f"Epoch [{epoch+1}/{num_epochs}], Train_Loss: {running_loss / len(loader):.4f}, Train_Accuracy: {accuracy:.4f}")

In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score
def test(model,test_loader, epoch):
    model.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Append predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate metrics
    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')

    wandb.log({"test_accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1, "Epoch" : epoch+1})

    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

In [20]:
def fit(model,optimizer,train_loader,test_loader):

    for epoch in range(num_epochs):
      train(model,optimizer,train_loader,epoch)
      test(model,test_loader,epoch)

In [21]:
num_epochs = 20

In [22]:
fit(shadow_model,shadow_optimizer,train_loader,test_loader)

Epoch [1/20], Train_Loss: 1.5595, Train_Accuracy: 47.3400
Test Accuracy: 60.80%
Precision: 0.6139
Recall: 0.6080
F1 Score: 0.5972
Epoch [2/20], Train_Loss: 1.0253, Train_Accuracy: 65.0400
Test Accuracy: 67.40%
Precision: 0.6802
Recall: 0.6740
F1 Score: 0.6706
Epoch [3/20], Train_Loss: 0.8586, Train_Accuracy: 70.4600
Test Accuracy: 71.95%
Precision: 0.7205
Recall: 0.7195
F1 Score: 0.7188
Epoch [4/20], Train_Loss: 0.7481, Train_Accuracy: 74.4300
Test Accuracy: 73.80%
Precision: 0.7363
Recall: 0.7380
F1 Score: 0.7342
Epoch [5/20], Train_Loss: 0.6515, Train_Accuracy: 77.7800
Test Accuracy: 75.35%
Precision: 0.7612
Recall: 0.7535
F1 Score: 0.7521
Epoch [6/20], Train_Loss: 0.5765, Train_Accuracy: 80.2400
Test Accuracy: 77.10%
Precision: 0.7712
Recall: 0.7710
F1 Score: 0.7675
Epoch [7/20], Train_Loss: 0.5286, Train_Accuracy: 82.3300
Test Accuracy: 76.70%
Precision: 0.7700
Recall: 0.7670
F1 Score: 0.7631
Epoch [8/20], Train_Loss: 0.4649, Train_Accuracy: 84.8000
Test Accuracy: 79.45%
Precision:

In [25]:
torch.save(shadow_model.state_dict(), 'shadow_model_5.pt')

In [26]:
test(shadow_model,train_loader, 1)

Test Accuracy: 98.70%
Precision: 0.9871
Recall: 0.9870
F1 Score: 0.9870
